In [15]:
import os
import numpy as np
import glob
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda, Reshape
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.optimizers import SGD, RMSprop, Adam

from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, concatenate, \
    Activation, ZeroPadding2D
from keras.layers import add, Flatten
from keras.utils import plot_model
from keras.metrics import top_k_categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, Lambda, Reshape, ZeroPadding2D
# from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# from tensorflow.keras.datasets import mnist
# from tensorflow.keras.utils import to_categorical

import cv2
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [51]:
def Conv2d_BN(x, nb_filter, kernel_size, strides=(1, 1), padding='same'):
    x = Conv2D(nb_filter, kernel_size, padding=padding, strides=strides, activation='relu')(x)
    print(x.shape)
    x = BatchNormalization(axis=1)(x)
    return x

def identity_Block(inpt, nb_filter, kernel_size, strides=(1, 1), with_conv_shortcut=False):
    x = Conv2d_BN(inpt, nb_filter=nb_filter, kernel_size=kernel_size, strides=strides, padding='same')
    x = Conv2d_BN(x, nb_filter=nb_filter, kernel_size=kernel_size, padding='same')
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt, nb_filter=nb_filter, strides=strides, kernel_size=kernel_size)
        x = add([x, shortcut])
        return x
    else:
        x = add([x, inpt])
        return x

def bottleneck_Block(inpt,nb_filters,strides=(1,1),with_conv_shortcut=False):
    k1,k2,k3=nb_filters
    x = Conv2d_BN(inpt, nb_filter=k1, kernel_size=1, strides=strides, padding='same')
    x = Conv2d_BN(x, nb_filter=k2, kernel_size=3, padding='same')
#     x = Dropout(0.2)(x)
    x = Conv2d_BN(x, nb_filter=k3, kernel_size=1, padding='same')
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt, nb_filter=k3, strides=strides, kernel_size=1)
        x = add([x, shortcut])
        return x
    else:
        x = add([x, inpt])
        return x

def resnet_50(width,height,channel,classes):
    inpt = Input(shape=(width, height, channel))
    x = ZeroPadding2D((2, 2))(inpt)
    x = Conv2d_BN(x, nb_filter=64, kernel_size=(6, 6), strides=(1, 1), padding='valid')
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

    #conv2_x
    x = bottleneck_Block(x, nb_filters=[64,64,256],strides=(1,1),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[64,64,256])
#     x = bottleneck_Block(x, nb_filters=[64,64,256])

    #conv3_x
    x = bottleneck_Block(x, nb_filters=[128, 128, 512],strides=(2,2),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[128, 128, 512])
#     x = bottleneck_Block(x, nb_filters=[128, 128, 512])
#     x = bottleneck_Block(x, nb_filters=[128, 128, 512])

    #conv4_x
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024],strides=(2,2),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
#     x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
#     x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
#     x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
#     x = bottleneck_Block(x, nb_filters=[256, 256, 1024])

    #conv5_x
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048], strides=(2, 2), with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048])
#     x = bottleneck_Block(x, nb_filters=[512, 512, 2048])

    x = AveragePooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(classes, activation='sigmoid')(x)

    model = Model(inputs=inpt, outputs=x)
    return model



In [52]:
model = resnet_50(69,69,3,37)
# model.summary()
model.compile(loss='mean_squared_error', optimizer=Adam(lr=1e-5))

(?, 68, 68, 64)
(?, 34, 34, 64)
(?, 34, 34, 64)
(?, 34, 34, 256)
(?, 34, 34, 256)
(?, 34, 34, 64)
(?, 34, 34, 64)
(?, 34, 34, 256)
(?, 17, 17, 128)
(?, 17, 17, 128)
(?, 17, 17, 512)
(?, 17, 17, 512)
(?, 17, 17, 128)
(?, 17, 17, 128)
(?, 17, 17, 512)
(?, 9, 9, 256)
(?, 9, 9, 256)
(?, 9, 9, 1024)
(?, 9, 9, 1024)
(?, 9, 9, 256)
(?, 9, 9, 256)
(?, 9, 9, 1024)
(?, 5, 5, 512)
(?, 5, 5, 512)
(?, 5, 5, 2048)
(?, 5, 5, 2048)
(?, 5, 5, 512)
(?, 5, 5, 512)
(?, 5, 5, 2048)


In [27]:
from random import shuffle
from scipy.misc import imresize
import csv

data_path = './'
TRAIN = './images_training_revtest/'
TEST = './images_test_rev1/'
LABELS = './training_solutions_rev1.csv'

class data_loader:    
    """
    Creates a class for handling train/valid/test data paths,
    training labels and image IDs.
    Useful for switching between sample and full datasets.
    """
    def __init__(self, path):    
        self.path = path 
        self.train_path = TRAIN
        #self.val_path = path + "valid"
        self.test_path = TEST
        
        def get_paths(directory):
            return [f for f in os.listdir(directory)]
        
        self.training_images_paths = get_paths(self.train_path)
        # self.validation_images_paths = get_paths(self.val_path)
        self.test_images_paths = get_paths(self.test_path)    
        
        def get_all_solutions():
        # Import solutions file and load into self.solutions
            all_solutions = {}
            # /'training_solutions_rev1.csv'
            with open(LABELS, 'r') as f:
                reader = csv.reader(f, delimiter=",")
                next(reader)
                for i, line in enumerate(reader):
                    all_solutions[line[0]] = [float(x) for x in line[1:]]
            return all_solutions
        
        self.all_solutions = get_all_solutions()

    def get_id(self,fname):
        return fname.replace(".jpg","").replace("data","")
        
    def find_label(self,val):
        return self.all_solutions[val]

def process_images(paths,aug=0):
    """
    Import image at 'paths', decode, centre crop and prepare for batching. 
    """
    count = len(paths)
    arr = np.zeros(shape=(count,3,69,69))
    for c, path in enumerate(paths):
        raw_img = cv2.imread(path).T  # 3,424,424
        img=raw_img
        img = img[:,108:315,108:315] #crop 424x424 -> 207x207
        img = imresize(img,size=(69,69,3),interp="cubic").T # downsample to half res
        arr[c] = img
    return arr


def BatchGenerator(getter):
    while 1:
        for f in getter.training_images_paths:
            X_train = process_images([getter.train_path + '/' + fname for fname in [f]])
            X_train = np.reshape(X_train,(1,69,69,3))
            id_ = getter.get_id(f)
            y_train = np.array(getter.find_label(id_))
            y_train = np.reshape(y_train,(1,37))
            assert(X_train.shape==(1,69,69,3))
            yield (X_train, y_train)


In [24]:
fetcher = data_loader(data_path)

In [25]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath='./weights.hdf5', verbose=1, save_best_only=True)

batch_size = 32
steps_to_take = int(len(fetcher.training_images_paths)*6/batch_size)
#val_steps_to_take = int(len(fetcher.validation_images_paths)/batch_size)
                #typically be equal to the number of unique samples if your dataset
                #divided by the batch size.

#model = load_model('tmp/weights.hdf5')

history = LossHistory()

In [36]:
hist = model.fit_generator(BatchGenerator(fetcher),
                    samples_per_epoch=steps_to_take, 
                    nb_epoch=10,
                    verbose=2,
                    callbacks=[history,checkpointer,early_stopping],
                   )

Epoch 1/10
 - 48s - loss: 0.0613
Epoch 2/10
 - 31s - loss: 0.0295
Epoch 3/10
 - 31s - loss: 0.0303
Epoch 4/10
 - 31s - loss: 0.0286
Epoch 5/10
 - 29s - loss: 0.0296
Epoch 6/10
 - 29s - loss: 0.0252
Epoch 7/10
 - 31s - loss: 0.0174
Epoch 8/10
 - 33s - loss: 0.0149
Epoch 9/10
 - 32s - loss: 0.0126
Epoch 10/10
 - 33s - loss: 0.0139


In [47]:
hist = model.fit_generator(BatchGenerator(fetcher),
                    samples_per_epoch=steps_to_take, 
                    nb_epoch=10,
                    verbose=2,
                    callbacks=[history,checkpointer,early_stopping],
                   )

Epoch 1/10
 - 40s - loss: 0.0695
Epoch 2/10
 - 25s - loss: 0.0290
Epoch 3/10
 - 26s - loss: 0.0298
Epoch 4/10
 - 25s - loss: 0.0291
Epoch 5/10
 - 26s - loss: 0.0292
Epoch 6/10
 - 26s - loss: 0.0283
Epoch 7/10
 - 27s - loss: 0.0263
Epoch 8/10
 - 27s - loss: 0.0266
Epoch 9/10
 - 27s - loss: 0.0272
Epoch 10/10
 - 27s - loss: 0.0281


In [50]:
hist = model.fit_generator(BatchGenerator(fetcher),
                    samples_per_epoch=steps_to_take, 
                    nb_epoch=10,
                    verbose=2,
                    callbacks=[history,checkpointer,early_stopping],
                   )

Epoch 1/10
 - 40s - loss: 0.0582
Epoch 2/10
 - 27s - loss: 0.0294
Epoch 3/10
 - 27s - loss: 0.0294
Epoch 4/10
 - 26s - loss: 0.0291
Epoch 5/10
 - 26s - loss: 0.0282
Epoch 6/10
 - 25s - loss: 0.0267
Epoch 7/10
 - 28s - loss: 0.0233
Epoch 8/10
 - 27s - loss: 0.0234
Epoch 9/10
 - 27s - loss: 0.0247
Epoch 10/10
 - 27s - loss: 0.0254
